In [1]:
import boto3 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LassoCV, Lasso
from sklearn.metrics import recall_score
from sklearn.preprocessing import MinMaxScaler

## defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data445-ashlyn-bucket'
bucket = s3.Bucket(bucket_name)

## defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## defining the input and target variables
X = heart[['age','male', 'currentSmoker', 'totChol','sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

Y.value_counts() / Y.shape[0]
## splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .2, stratify = Y)

In [3]:
## transforming to 0-1 scale
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

In [4]:
X_train.head()

,age,male,currentSmoker,totChol,sysBP,BMI,heartRate,glucose
0,0.157895,0.0,1.0,0.402464,0.139480,0.140814,0.161616,0.098870
1,0.526316,0.0,0.0,0.312115,0.255319,0.275570,0.282828,0.129944
2,0.526316,0.0,0.0,0.312115,0.219858,0.279932,0.191919,0.081921
3,0.289474,0.0,0.0,0.400411,0.125296,0.225158,0.111111,0.121469
4,0.736842,1.0,0.0,0.143737,0.144208,0.194377,0.313131,0.053672


In [5]:
## LASSO
lasso_cv = LassoCV(cv = 5).fit(X_train, Y_train)
lasso_cv.alpha_

## building lasso regression
lasso_md = Lasso(alpha = lasso_cv.alpha_).fit(X_train, Y_train)
lasso_md.coef_

array([ 0.27502386,  0.06602008,  0.04498047,  0.02544067,  0.54443588,
        0.        , -0.07112773,  0.41741261])

In [6]:
X_train_lasso = X_train[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]
X_test_lasso = X_test[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]

##logistic regression
logit_md = LogisticRegression().fit(X_train_lasso, Y_train)

##predicting on test
pred1 = logit_md.predict_proba(X_test_lasso)[:, 1]
pred1 = np.where(pred1 < 0.1, 0, 1)

## computing the recall
recall_score(Y_test, pred1)

0.8303571428571429

In [7]:
##Random forest classifier
RF = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)

##extracting importance
RF.feature_importances_

array([0.14807264, 0.02294575, 0.02127552, 0.16203997, 0.18932724,
       0.17350123, 0.12292785, 0.1599098 ])

In [9]:
X_train_rf = X_train[['totChol', 'male', 'currentSmoker', 'sysBP', 'BMI']]
X_test_rf = X_test[['totChol', 'male', 'currentSmoker', 'sysBP', 'BMI']]

##building the random forest model
RF = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_rf, Y_train)

##predicting on test
pred2 = RF.predict_proba(X_test_rf)[:, 1]
pred2 = np.where(pred2 < 0.1, 0, 1)

##computing the recall score
recall_score(Y_test, pred2)

0.9553571428571429